In [1]:
%matplotlib inline
if 1:
    import matplotlib
    matplotlib.use("pgf")
    matplotlib.rcParams.update({
        "pgf.texsystem": "pdflatex",
        'font.family': 'serif',
        'text.usetex': True,
        'pgf.rcfonts': False,
    })
import matplotlib.pyplot as plt
import pickle
import numpy as np
from pathlib import Path
from utils import getsize
import pandas as pd
from collections import OrderedDict

In [2]:
files = Path('/home/guso/Data/lemsim/')

In [3]:
data = {}
for fl in files.glob('*?test5.pkl'):
    with open(fl, 'rb') as fh: data[str(fl)] = pickle.load(fh)

In [4]:
KEYS = list(data.keys())
KEYS = sorted(KEYS)

N = len(data[KEYS[0]][1])
L = data[KEYS[0]][1][0]['L']

df = pd.DataFrame([k.replace('--', '-')[23:-10].split('-') for k in KEYS])
df.columns = ['N', 'T', 'D', 'pt', 'market', 'freq', 'seed', 'only_update_price', 'flat', 'start']

df.flat = df.flat.fillna(False)
df.only_update_price = df.only_update_price.fillna(False)

for col in ['N', 'T', 'D', 'seed', 'start']:
    df[col] = df[col].astype(int)
    
for col in ['only_update_price', 'flat', 'market']:
    df[col] = df[col].map(lambda x: True if x=='True' else False)

In [5]:
def get_default_key(num, df):
    tmp = df.loc[num]
    default = df[
        (df.seed == tmp.seed) &
        (df.start == tmp.start) &
        (df.flat == tmp.flat) &
        (df.market == False)
    ].index[0]
    
    return default
    
#get_default_key(41, df)

In [6]:
def social_welfare(num):
    """
    Calculates the social welfare for a the key `num`
    """
    key = KEYS[num]
    default_key = KEYS[get_default_key(num, df)]
    
    dt = data[key][1]
    dt_default = data[default_key][1]
    
    
    total_cost = np.array([dt[n]['history_cost'].sum() for n in range(N)]).sum().round()
    default_cost = np.array([dt_default[n]['history_cost'].sum() for n in range(N)]).sum().round()
    
    #print(key, total_cost, default_cost)
    
    res = ((total_cost - default_cost) / default_cost) * 100
    
    return res

def increase_in_range(num, df):
    key = KEYS[num]
    default_key = KEYS[get_default_key(num, df)]
    
    dt = data[key][1]
    dt_default = data[default_key][1]
    
    

    net_load = np.vstack([dt[n]['history_post_net'] for n in range(N)]).sum(axis=0)[: L - 47]
    default_load = np.vstack([dt_default[n]['history_post_net'] for n in range(N)]).sum(axis=0)[: L - 47]
    nl_max, nl_min = net_load.max(), net_load.min()
    dl_max, dl_min = default_load.max(), default_load.min()
    
    #print('New', nl_min, nl_max)
    #print('Def', dl_min, dl_max)
    
    top_range = round(((nl_max - dl_max) / np.abs(dl_max)) * 100, 2)
    bottom_range = round(((dl_min - nl_min) / np.abs(nl_min)) * 100, 2)
    
    bottom_range = bottom_range if np.abs(bottom_range) < 1e4 else None
    top_range = top_range if np.abs(top_range) < 1e4 else None
    
    
    return bottom_range, top_range 

def matched_local(num, df):
    
    key = KEYS[num]
    default_key = KEYS[get_default_key(num, df)]
    
    dt = data[key][1]
    dt_default = data[default_key][1]

    l = np.vstack([dt[n]['history_post_net'] for n in range(N)])[:, : L - 47]
    d = np.vstack([dt_default[n]['history_post_net'] for n in range(N)])[: L - 47]
    
    l_p = np.clip(l, a_max=1e7, a_min=0)
    l_n = np.clip(-l, a_max=1e7, a_min=0)

    d_p = np.clip(d, a_max=1e7, a_min=0)
    d_n = np.clip(-d, a_max=1e7, a_min=0)
    
    net_matched = np.minimum(l_n.sum(axis=0), l_p.sum(axis=0)).sum()
    default_matched = np.minimum(d_n.sum(axis=0), d_p.sum(axis=0)).sum()
    
    inc = np.round(((net_matched - default_matched) / np.abs(default_matched)) * 100 , 2)
    
    return inc

def matched_local_net(num, df):
    
    key = KEYS[num]
    default_key = KEYS[get_default_key(num, df)]
    
    dt = data[key][1]
    dt_default = data[default_key][1]

    l = np.vstack([dt[n]['history_post_net'] for n in range(N)])[:, : L - 47]
    d = np.vstack([dt_default[n]['allload'] for n in range(N)])[: L - 47]
    
    l_p = np.clip(l, a_max=1e7, a_min=0)
    l_n = np.clip(-l, a_max=1e7, a_min=0)

    d_p = np.clip(d, a_max=1e7, a_min=0)
    d_n = np.clip(-d, a_max=1e7, a_min=0)
    
    net_matched = np.minimum(l_n.sum(axis=0), l_p.sum(axis=0)).sum()
    default_matched = np.minimum(d_n.sum(axis=0), d_p.sum(axis=0)).sum()
    
    inc = np.round(((net_matched - default_matched) / np.abs(default_matched)) * 100 , 2)
    
    return net_matched
    

matched_local_net(1, df)

113.73215190475848

In [7]:
df['social_cost'] = df.apply(lambda x: social_welfare(x.name), axis=1)
df['increase_bottom_range'] = df.apply(lambda x: increase_in_range(x.name, df)[0], axis=1)
df['increase_top_range'] = df.apply(lambda x: increase_in_range(x.name, df)[1], axis=1)
df['local_matched'] = df.apply(lambda x: matched_local(x.name, df), axis=1)
df['local_matched_net'] = df.apply(lambda x: matched_local_net(x.name, df), axis=1)

In [8]:
df

,N,T,D,pt,market,freq,seed,only_update_price,flat,start,social_cost,increase_bottom_range,increase_top_range,local_matched,local_matched_net
0,50,48,10,neutral,True,1,101,False,False,130,-1.592899,-73.58,60.12,1.16,169.526849
1,50,48,10,neutral,True,1,101,False,True,130,-3.070529,12.77,1.36,-15.53,113.732152
2,50,48,10,neutral,True,1,101,True,False,130,-1.203994,-25.63,64.24,-24.76,126.081311
3,50,48,10,neutral,True,1,101,True,True,130,-2.978933,12.28,1.36,-22.44,104.429636
4,50,48,10,neutral,True,1,1234,False,False,100,-1.119678,-37.39,145.40,-13.91,156.082834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,50,48,10,unique,True,5,404,True,True,160,1.664503,71.16,-3.75,52.73,319.868548
304,50,48,10,unique,True,5,505,False,False,170,0.146703,8.92,4.83,4.47,234.918081
305,50,48,10,unique,True,5,505,False,True,170,-1.251747,-6.23,-5.40,-1.49,224.866217
306,50,48,10,unique,True,5,505,True,False,170,16.009679,77.00,-13.35,103.26,457.049485


In [9]:
dist2name = {
    'neutral': 'NEU',
    'optimistic': 'OPT',
    'pesimistic': 'PES',
    'solar': 'SOL',
    'unique': 'UNQ'
}

df_ = df.copy()
METRICS = ['social_cost', 'increase_bottom_range', 'increase_top_range', 'local_matched', 'local_matched_net']
df_['Tariff'] = df_['flat'].map({False: 'TOU', True: 'Flat'})
df_['Belief'] = df_['pt'].map(dist2name)
df_['Update Q'] = ~ df_['only_update_price']
df_['Freq'] = df_['freq']
df_ = df_[(df_['Update Q'] == True) | (df_['Freq'] == None)]
df_ = df_.drop(['N', 'T', 'D', 'seed', 'start', 'pt', 'flat', 'only_update_price', 'freq', 'Update Q'], axis=1)


summary = df_.groupby(['Tariff', 'Freq', 'Belief'])[METRICS].mean()
summary[METRICS] = summary[METRICS].astype(int)
summary.columns = ['SC', 'min', 'max', 'LM', 'net LM']
#summar.astype(int)

In [10]:
summary

SC  min  max  LM  net LM
Tariff Freq Belief                          
Flat   1    NEU     -3  -64    0  -1     206
            OPT     -4  -90    0   3     216
            PES     -2  -38    0   2     211
            SOL     -3  -62    0  19     247
            UNQ     -3  -58   -1  27     261
       5    NEU     -3    1    1  -4     202
            OPT     -4  -64    1  11     232
            PES     -1   -2    0   0     209
            SOL     -1   -7    0   6     221
            UNQ     -1    7   -3   0     210
       None PES      0    0    0   0     208
            SOL    -12  -32  121  33     274
TOU    1    NEU     -1  -31   75 -13     203
            OPT      6 -120  -30  48     346
            PES      0  -22   37  -9     214
            SOL     -1  -24   21  28     301
            UNQ      0  -44    8  36     321
       5    NEU     -1   20   76 -13     205
            OPT      6  -85  -30  72     399
            PES      0    1   37   0     235
            SOL      0   -5   21   2     241
            UNQ      0    7  -21   4     245
       None PES      0    0    0   0     234
            SOL     -8 -237   -5  29     302

In [11]:
with open('img/table_2.tex', 'w') as fh:
    fh.write(summary.to_latex(escape=False))
    
summary

SC  min  max  LM  net LM
Tariff Freq Belief                          
Flat   1    NEU     -3  -64    0  -1     206
            OPT     -4  -90    0   3     216
            PES     -2  -38    0   2     211
            SOL     -3  -62    0  19     247
            UNQ     -3  -58   -1  27     261
       5    NEU     -3    1    1  -4     202
            OPT     -4  -64    1  11     232
            PES     -1   -2    0   0     209
            SOL     -1   -7    0   6     221
            UNQ     -1    7   -3   0     210
       None PES      0    0    0   0     208
            SOL    -12  -32  121  33     274
TOU    1    NEU     -1  -31   75 -13     203
            OPT      6 -120  -30  48     346
            PES      0  -22   37  -9     214
            SOL     -1  -24   21  28     301
            UNQ      0  -44    8  36     321
       5    NEU     -1   20   76 -13     205
            OPT      6  -85  -30  72     399
            PES      0    1   37   0     235
            SOL      0   -5   21   2     241
            UNQ      0    7  -21   4     245
       None PES      0    0    0   0     234
            SOL     -8 -237   -5  29     302

In [12]:
def basic_plot(seed, update_q, tariff, start):
    df_ = df[
        (df.seed == seed) & ((df.only_update_price == update_q) | (df.freq == 'None')) & (df.flat == tariff) & (df.start == start)
    ].copy()
    
    def_key = df_[(df_.pt == 'pesimistic') & (df_.market == False)].index[0]
    dt = data[KEYS[def_key]][1]
    defload = np.vstack([dt[n]['history_post_net'] for n in range(N)]).sum(axis=0)[: L - 47]
    prices_b = dt[0]['allprices'][:, 3]
    prices_s = dt[0]['allprices'][:, 0]
    
    fig = plt.figure(figsize=(14, 10), constrained_layout=True)
    ax = fig.add_subplot(111)
    
    for l in range(L - 47):
        color = 'y' if prices_b[l] > prices_b.min() else 'g'
        ax.axvspan(l, l+1, facecolor=color, alpha=0.3)
    
    

    for i, rows in df_.iterrows():
        dt = data[KEYS[i]][1]
        if rows.pt == 'solar' and rows.market is False:
            color = 'k'
            width = 1
            label = 'Solar Tariff'
        elif rows.pt == 'pesimistic' and rows.market is False:
            continue
            #color = 'k'
            #width = 1
            #label = 'Solar Tariff'
        else:
            color = 'r'
            width = 0.5
            label = 'Other beliefs'
        netload = np.vstack([dt[n]['history_post_net'] for n in range(N)]).sum(axis=0)[: L - 47]
        ax.plot(netload - defload, color=color, linewidth=width, label=label)
    ax.legend()
    
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True)
    
    name = 'img/{}_{}_{}_{}_2'.format(seed, update_q, tariff, start)
    

    
    ax.set_xlabel('Time-slots')
    ax.set_ylabel('Net energy consumption (kWh)')
    
    ax.set_xlim(0, L - 47)
    
    #ax.legend(l
    
    handles, labels = fig.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper center', bbox_to_anchor=(.5, 1.3),
          ncol=2, fancybox=True, shadow=True)
    
    phi = (1 + np.sqrt(5)) * 0.5
    fig.set_size_inches(3.5, 3.5 / phi)
    fig.tight_layout(pad=0)
    fig.savefig(name + '.pgf')
    fig.show()
    
    return df_

In [13]:
df_ = basic_plot(seed=101, update_q=True, tariff=False, start=130)

/home/guso/github/lemsim_code/venv/lib/python3.7/site-packages/ipykernel_launcher.py:61: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
/home/guso/github/lemsim_code/venv/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.


In [14]:
df_ = basic_plot(seed=101, update_q=True, tariff=True, start=130)

/home/guso/github/lemsim_code/venv/lib/python3.7/site-packages/ipykernel_launcher.py:61: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
/home/guso/github/lemsim_code/venv/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.


In [15]:
def pre_post_plot(seed, update_q, tariff, start, pt, freq):
    df_ = df[
        (df.seed == seed) & ((df.only_update_price == update_q) | (df.freq == 'None')) &
        (df.flat == tariff) & (df.start == start) & (df.pt == pt) & (df.freq == str(freq))
    ].copy()
    
    key = df_.index[0]
    dt = data[KEYS[key]][1]
    
    #def_key = df_[(df_.pt == 'pesimistic') & (df_.market == False)].index[0]
    #
    #defload = np.vstack([dt[n]['history_post_net'] for n in range(N)]).sum(axis=0)[: L - 47]
    prices_b = dt[0]['allprices'][:, 3]
    prices_s = dt[0]['allprices'][:, 0]
    
    pre_load = np.vstack([dt[n]['history_pre_net'] for n in range(N)]).sum(axis=0)[: L - 47]
    post_load = np.vstack([dt[n]['history_post_net'] for n in range(N)]).sum(axis=0)[: L - 47]
    
    #return df_
    
    fig = plt.figure(figsize=(14, 10), constrained_layout=True)
    ax = fig.add_subplot(111)
    
    for l in range(L - 47):
        color = 'y' if prices_b[l] > prices_b.min() else 'g'
        ax.axvspan(l, l+1, facecolor=color, alpha=0.3)
    

    ax.plot(pre_load, label='Desired Load', c='r')
    ax.plot(post_load, label='Consumed Load', c='b')
    
    ax.legend()
    
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True)
    
    name = 'img/{}_{}_{}_{}_{}_{}_pre_post_2'.format(seed, update_q, tariff, start, pt, freq)
    

    
    ax.set_xlabel('Time-slots')
    ax.set_ylabel('Net energy consumption (kWh)')
    
    ax.set_xlim(0, L - 47)

    handles, labels = fig.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper center', bbox_to_anchor=(.5, 1.3),
          ncol=2, fancybox=True, shadow=True)
    
    phi = (1 + np.sqrt(5)) * 0.5
    fig.set_size_inches(3.5, 3.5 / phi)
    fig.tight_layout(pad=0)
    fig.savefig(name + '.pgf')
    fig.show()
    
    return df_

In [16]:
pre_post_plot(seed=101, update_q=True, tariff=False, start=130, pt='neutral', freq=1)

/home/guso/github/lemsim_code/venv/lib/python3.7/site-packages/ipykernel_launcher.py:53: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
/home/guso/github/lemsim_code/venv/lib/python3.7/site-packages/ipykernel_launcher.py:55: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.


,N,T,D,pt,market,freq,seed,only_update_price,flat,start,social_cost,increase_bottom_range,increase_top_range,local_matched,local_matched_net
2,50,48,10,neutral,True,1,101,True,False,130,-1.203994,-25.63,64.24,-24.76,126.081311


In [17]:
df

,N,T,D,pt,market,freq,seed,only_update_price,flat,start,social_cost,increase_bottom_range,increase_top_range,local_matched,local_matched_net
0,50,48,10,neutral,True,1,101,False,False,130,-1.592899,-73.58,60.12,1.16,169.526849
1,50,48,10,neutral,True,1,101,False,True,130,-3.070529,12.77,1.36,-15.53,113.732152
2,50,48,10,neutral,True,1,101,True,False,130,-1.203994,-25.63,64.24,-24.76,126.081311
3,50,48,10,neutral,True,1,101,True,True,130,-2.978933,12.28,1.36,-22.44,104.429636
4,50,48,10,neutral,True,1,1234,False,False,100,-1.119678,-37.39,145.40,-13.91,156.082834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,50,48,10,unique,True,5,404,True,True,160,1.664503,71.16,-3.75,52.73,319.868548
304,50,48,10,unique,True,5,505,False,False,170,0.146703,8.92,4.83,4.47,234.918081
305,50,48,10,unique,True,5,505,False,True,170,-1.251747,-6.23,-5.40,-1.49,224.866217
306,50,48,10,unique,True,5,505,True,False,170,16.009679,77.00,-13.35,103.26,457.049485


In [18]:
dt = data[KEYS[0]][1]
np.sum([dt[n]['history_cost'].sum() for n in range(N)])

97671.90233142038

In [19]:
dt2 = data[KEYS[get_default_key(0, df)]][1]
np.sum([dt2[n]['history_cost'].sum() for n in range(N)])

99253.32085053665

In [20]:
(97662.42083654413 - 99096.44541109315) / 99096.44541109315 * 100

-1.447099912211882

In [21]:
df.loc[80]

N                                50
T                                48
D                                10
pt                       optimistic
market                         True
freq                              1
seed                            505
only_update_price             False
flat                          False
start                           170
social_cost                 7.83242
increase_bottom_range       -166.37
increase_top_range           -13.53
local_matched                 37.34
local_matched_net           308.827
Name: 80, dtype: object

In [22]:
increase_in_range(4, df)

(-37.39, 145.4)